# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading CSV file that was created in Part I
# Create weather DataFrame
weather_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Piteå,65.32,21.48,51.80,100,92,5.82,SE,1601506948
1,Port-Gentil,-0.72,8.78,77.00,83,20,6.93,GA,1601506421
2,Nome,64.50,-165.41,48.99,57,1,25.28,US,1601506558
3,Pacific Grove,36.62,-121.92,86.00,62,1,6.93,US,1601506949
4,Xiongzhou,25.12,114.30,70.54,85,72,1.92,CN,1601506949
...,...,...,...,...,...,...,...,...,...
550,Takoradze,4.88,-1.76,76.05,89,89,9.37,GH,1601506692
551,Magnolia,33.27,-93.24,84.20,35,1,8.05,US,1601507115
552,Sangar,63.92,127.47,34.16,85,100,4.21,RU,1601507116
553,Gushikawa,26.35,127.87,73.40,73,1,4.70,JP,1601507116


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations 
locations = weather_data_df[["Lat", "Lng"]]

# Use Humidity as the weight
humidity = weather_data_df["Humidity"]

In [4]:
#
#  Add Heatmap layer to map.
#
# Get gamps for display
#
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

max_intensity = np.max(humidity)

# Create heat layer based on locations, humidity and max humidity 
#
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add heat layer for the display
fig.add_layer(heat_layer)

# Display the Humidity Hit Map now
#
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create new DataFrame fitting weather criteria:
# Narrow down the cities with wind speed less than 10 mph, 
# cloudiness equals to 0 and max temp between 60 and 80
#
narrowed_city_df = weather_data_df.loc[(weather_data_df["Wind Speed"] <= 10) & \
                                       (weather_data_df["Cloudiness"] == 0) & \
                                       (weather_data_df["Max Temp"] >= 70) & \
                                       (weather_data_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,Kiryat Gat,31.61,34.76,78.01,81,0,1.59,IL,1601506464
225,Jalu,29.03,21.55,79.54,38,0,9.13,LY,1601506697
243,Karratha,-20.74,116.85,76.32,35,0,4.94,AU,1601507015
274,Angoche,-16.23,39.91,73.74,89,0,4.92,MZ,1601507024
305,Riyadh,24.69,46.72,78.80,24,0,3.36,SA,1601507034
357,Bulungu,-4.55,18.60,73.20,84,0,0.63,CD,1601507051
369,Mitsamiouli,-11.38,43.28,78.40,77,0,5.03,KM,1601507055
373,Bhilwara,25.35,74.63,75.45,54,0,7.18,IN,1601507056
408,Enumclaw,47.20,-121.99,77.00,65,0,4.03,US,1601507069
471,Kānt,27.80,79.80,73.69,59,0,3.33,IN,1601507087


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Creating a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Adding a "Hotel Name" column to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display DataFrame
hotel_df


,City,Country,Lat,Lng,Hotel Name
151,Kiryat Gat,IL,31.61,34.76,
225,Jalu,LY,29.03,21.55,
243,Karratha,AU,-20.74,116.85,
274,Angoche,MZ,-16.23,39.91,
305,Riyadh,SA,24.69,46.72,
357,Bulungu,CD,-4.55,18.60,
369,Mitsamiouli,KM,-11.38,43.28,
373,Bhilwara,IN,25.35,74.63,
408,Enumclaw,US,47.20,-121.99,
471,Kānt,IN,27.80,79.80,


In [7]:
# Use base_url for hotel search 
#
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [8]:
# This loop performs the follwoing tasks:
# 1. Hit the Google Places API for each city's coordinates.
# 2. Store the first Hotel result into the DataFrame.
# 3. Plot markers on top of the heatmap.
#
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    city_name = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # create dictionary for search params with location based on lattitude and 
    # logitude
    #
    params["location"] = f"{lat},{lng}"

    # Perform API request from Google places API for city_name based on params
    #
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Get repsonse results
    #
    results = response['results']
    
    # 
    # Store the first Hotel result into the DataFrame
    # 
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # 
    # If there is not hotel available then print as Missing field/result ...
    #
    except (KeyError, IndexError):
        print("Missing field/result ... skipping ")
        
    print("--------------------------------------------")
    
    # Wait 1 sec to before next api call in order to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("--------------------------End of Hotel Search--------------------------")


Retrieving Results for Index 151: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
--------------------------------------------
Retrieving Results for Index 225: Jalu.
Closest hotel in Jalu is Jalu Hotel.
--------------------------------------------
Retrieving Results for Index 243: Karratha.
Closest hotel in Karratha is Karratha International Hotel.
--------------------------------------------
Retrieving Results for Index 274: Angoche.
Closest hotel in Angoche is Guest House HESADA.
--------------------------------------------
Retrieving Results for Index 305: Riyadh.
Closest hotel in Riyadh is Four Seasons Hotel Riyadh At Kingdom Center.
--------------------------------------------
Retrieving Results for Index 357: Bulungu.
Missing field/result ... skipping 
--------------------------------------------
Retrieving Results for Index 369: Mitsamiouli.
Closest hotel in Mitsamiouli is Al Camar Lodge.
--------------------------------------------
Retrieving Results for Index 373: Bhil

In [9]:
# Display the hotel dataframe
hotel_df


,City,Country,Lat,Lng,Hotel Name
151,Kiryat Gat,IL,31.61,34.76,Desert Gat
225,Jalu,LY,29.03,21.55,Jalu Hotel
243,Karratha,AU,-20.74,116.85,Karratha International Hotel
274,Angoche,MZ,-16.23,39.91,Guest House HESADA
305,Riyadh,SA,24.69,46.72,Four Seasons Hotel Riyadh At Kingdom Center
357,Bulungu,CD,-4.55,18.60,
369,Mitsamiouli,KM,-11.38,43.28,Al Camar Lodge
373,Bhilwara,IN,25.35,74.63,Ranbanka Heritage Resort Bhilwara
408,Enumclaw,US,47.20,-121.99,GuestHouse Enumclaw
471,Kānt,IN,27.80,79.80,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))